In [76]:
import re
import math
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score ,precision_score , recall_score , confusion_matrix , f1_score
from sklearn.model_selection import GridSearchCV
from ast import literal_eval
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import PCA
pca = PCA(n_components=40)

In [77]:
eucl_data = pd.read_csv('../../analysis/cluster/twitter/(test)_twitter_2020_up2_euclidean.csv', error_bad_lines=False)
eucl_data['closer#'] = 2 # cosine 1 , euclidean 2

cosine_data = pd.read_csv('../../analysis/cluster/twitter/(test)_twitter_2020_up2_cosine.csv', error_bad_lines=False)
cosine_data['closer#'] = 1 # cosine 1 , euclidean 2

data_2020U2 = eucl_data
data_2020U2.append(cosine_data)

data_2020U2['period'] = 202012 # 2020년의 up 1 , 첫번째 1

data_2020U2['Silhouette'] = 0.009771409
data_2020U2['result'] = 1



In [78]:
data_2020D2.isnull().sum()

Unnamed: 0    0
raw_data      0
word          0
cluster       0
centroid      0
distance      0
tfidf         0
score         0
closer#       0
category      0
label         0
result        0
Silhouette    0
period        0
dtype: int64

In [79]:
eucl_data = pd.read_csv('../../analysis/cluster/twitter/(test)_twitter_2020_down2_euclidean.csv', error_bad_lines=False)
eucl_data['closer#'] = 2 # cosine 1 , euclidean 2

cosine_data = pd.read_csv('../../analysis/cluster/twitter/(test)_twitter_2020_down2_cosine.csv', error_bad_lines=False)
cosine_data['closer#'] = 1 # cosine 1 , euclidean 2

data_2020D2 = eucl_data
data_2020D2.append(cosine_data)

data_2020D2['period'] = 202022 # 2020년의 down 2 , 첫번째 1



In [80]:
data_2020D2['result'].isnull().sum()

0

In [81]:
data_train = pd.concat([data_2020U2,data_2020D2],ignore_index=True)

/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [82]:
data_train['result']

0       1
1       1
2       1
3       1
4       1
       ..
2442    2
2443    2
2444    2
2445    2
2446    2
Name: result, Length: 2447, dtype: int64

In [83]:
data_train['period'] = pd.to_numeric(data_train['period'], errors='coerce')
data_train['category'] = pd.to_numeric(data_train['category'], errors='coerce')
data_train['label'] = pd.to_numeric(data_train['label'], errors='coerce')
data_train['Silhouette'] = pd.to_numeric(data_train['Silhouette'], errors='coerce')

In [84]:
x_data_df = data_train[['period','raw_data','closer#','category','label','Silhouette']]
y_data_df = data_train['result']

In [85]:
header = []
for i in range(1,101) :
    header.append('C'+str(i))

In [86]:
x_data_df['raw_data'] = x_data_df['raw_data'].apply(lambda x: x[1:len(x)-1].split(','))
print(len(x_data_df)) 

2447


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
tempFrame = pd.DataFrame(columns = header )

In [88]:
for index,row in x_data_df.iterrows() :

    raw_data_list = [float(i.replace(" ","")) for i in row['raw_data']]
    #print(raw_data_list)
    #print(len(raw_data_list))
    #print(len(header))
    
    
    tempSeries = pd.Series(raw_data_list , index = header)
    
    tempFrame = tempFrame.append(tempSeries , ignore_index=True)
    
    if( (index%1000)==0 ) :
        print(index)
    
    #for k in range(0,len(raw_data_list1)) :
    #    col = 'C'+str(k+1)
        #print(col)
        #row(col) = raw_data_list1[k]
    #    x_data_df.loc[index,col] = raw_data_list1[k]

0
1000
2000


In [89]:
x_data_df = pd.concat([x_data_df,tempFrame],axis=1,ignore_index=False)

In [90]:
#x_data_df['raw_data'] = x_data_df['raw_data'].apply(lambda x : np.mean(np.array([i for i in x],dtype=float)) )
# x_data_df['raw_data'] = pd.to_numeric(x_data_df['raw_data'],errors='coerce')

#x_data_df['raw_data'] = x_data_df['raw_data'].convert_objects(convert_numeric=True)
#x_data_df['raw_data'] = x_data_df['raw_data'].astype(float)

In [91]:
x_data_df = x_data_df.drop('raw_data',axis=1)

In [92]:
x_data_df.isnull().sum()

period        0
closer#       0
category      0
label         0
Silhouette    0
             ..
C96           0
C97           0
C98           0
C99           0
C100          0
Length: 105, dtype: int64

In [93]:
X_train,X_test, y_train, y_test = train_test_split(x_data_df,y_data_df,test_size=0.2,random_state=11)

In [94]:
rf_clf = RandomForestClassifier(random_state = 11 ,n_jobs=-1) # random forest

In [95]:
y_train

1732    2
272     1
1820    2
1126    1
1108    1
       ..
1697    2
332     1
1293    1
1104    1
1945    2
Name: result, Length: 1957, dtype: int64

In [96]:
rf_clf.fit(X_train,y_train) 

RandomForestClassifier(n_jobs=-1, random_state=11)

In [97]:
rf_pred = rf_clf.predict(X_test)  # random forest predict

In [98]:
print('Random Forest Accuracy :  ',accuracy_score(y_test,rf_pred))
print('Random Forest Precision :  ',precision_score(y_test,rf_pred,pos_label='positive',average='weighted'))
print('Random Forest Recall :  ',recall_score(y_test,rf_pred,pos_label='positive',average='weighted'))
print('Random Forest f1 :  ',f1_score(y_test,rf_pred,pos_label='positive',average='weighted'))

Random Forest Accuracy :   1.0
Random Forest Precision :   1.0
Random Forest Recall :   1.0
Random Forest f1 :   1.0


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [99]:
dt_clf = DecisionTreeClassifier(random_state = 11)
dt_clf.fit(X_train,y_train)
dt_pred = dt_clf.predict(X_test)

In [100]:
print('Decision Tree Accuracy :  ',accuracy_score(y_test,dt_pred))
print('Decision Tree Precision :  ',precision_score(y_test,dt_pred,pos_label='positive',average='weighted'))
print('Decision Tree Recall :  ',recall_score(y_test,dt_pred,pos_label='positive',average='weighted'))
print('Decision Tree f1 :  ',f1_score(y_test,dt_pred,pos_label='positive',average='weighted'))

Decision Tree Accuracy :   1.0
Decision Tree Precision :   1.0
Decision Tree Recall :   1.0
Decision Tree f1 :   1.0


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [101]:
gb_clf = GradientBoostingClassifier(random_state = 11) # random forest

In [102]:
gb_clf.fit(X_train,y_train)
gb_pred = gb_clf.predict(X_test)

In [103]:
print('Gredient Boosting Accuracy :  ',accuracy_score(y_test,gb_pred))
print('Gredient Boosting Precision :  ',precision_score(y_test,gb_pred,pos_label='positive',average='weighted'))
print('Gredient Boosting Recall :  ',recall_score(y_test,gb_pred,pos_label='positive',average='weighted'))
print('Gredient Boosting f1 :  ',f1_score(y_test,gb_pred,pos_label='positive',average='weighted'))

Gredient Boosting Accuracy :   1.0
Gredient Boosting Precision :   1.0
Gredient Boosting Recall :   1.0
Gredient Boosting f1 :   1.0


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [104]:
lr_clf = LogisticRegression(random_state = 11)

In [105]:
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)

In [106]:
print('Logistic Regression Accuracy :  ',accuracy_score(y_test,lr_pred))
print('Logistic Regression  Precision :  ',precision_score(y_test,lr_pred,pos_label='positive',average='weighted'))
print('Logistic Regression  Recall :  ',recall_score(y_test,lr_pred,pos_label='positive',average='weighted'))
print('Logistic Regression  f1 :  ',f1_score(y_test,lr_pred,pos_label='positive',average='weighted'))

Logistic Regression Accuracy :   0.6857142857142857
Logistic Regression  Precision :   0.4702040816326531
Logistic Regression  Recall :   0.6857142857142857
Logistic Regression  f1 :   0.5578692493946732


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [108]:
#x_data_df.to_csv('./data/twitter_data_before_analysis1.csv')
data_df

NameError: name 'data_df' is not defined

In [109]:
data_df = pd.merge(x_data_df,y_data_df,left_index=True,right_index=True)
#data_df.to_csv("./data/ko_news_data_before_analysis1.csv")
#data_df.to_csv("./testdata/twitter_analysis.csv")

In [110]:
#data_df.to_csv("./testdata/twitter_analysis.csv")
#data_df.to_csv("./data/twitter_data_before_analysis1.csv")
data_df.to_csv("./data/(test)_twitter_data_before_analysis.csv")